In [ ]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

indexes = ['Propin1', 'SBER1', 'NR1', 'AMAWproc', 'AAPENproc', 'AMINCproc',	
           'DJNU', 'REINB', 'Pmproc', 'LPOORproc', 'LPOOR', 'KBIRTS', 'DEATHS', 'KDEATHS', 
           'ND', 'KND', 'MIGRAOUT', 'SMIGRA', 'EAP', 'EMECproc', 'UNEM', 'UNEMproc', 'UNEMREG']
num_neurons = [15, 20, 25, 30]
num_epochs = [60, 100]
act_fun = ['relu', 'elu', 'selu', 'sigmoid']
column_names = ['Neurons', 'Epochs', 'Function', 'MAPE', 'Prediction']


for l in indexes:
    
    result = pd.DataFrame(columns = column_names)
    result_list = []
    dataset = pd.ExcelFile('datasets.xlsx').parse(str(l))
    train_dataset = dataset[:31]
    test_dataset = dataset.drop(train_dataset.index)
    train_stats = train_dataset.describe()
    train_stats_label=train_stats.pop(str(l))
    train_stats = train_stats.transpose()
    train_labels = train_dataset.pop(str(l))
    test_labels = test_dataset.pop(str(l))
    max_val_x = max(train_dataset.squeeze())
    min_val_x = min(train_dataset.squeeze())
    max_val_y = max(train_labels)
    min_val_y = min(train_labels)

    def norm_x(x):
        return (x - min_val_x) / (max_val_x - min_val_x)
    def norm_y(y):
        return (y - min_val_y) / (max_val_y - min_val_y)

    normed_train_data = norm_x(train_dataset)
    normed_test_data = norm_x(test_dataset)
    normed_train_labels = norm_y(train_labels)
    normed_test_labels = norm_y(test_labels)

    normed_train_dataset = norm_x(train_dataset)
    
    for k in act_fun:
        for j in num_epochs:
            for i in num_neurons:
                def build_model():
                    model = keras.Sequential([
                        layers.Dense(i, activation=k, input_shape=[len(normed_train_dataset.keys())]),
                        layers.Dense(1)])
                    optimizer = tf.keras.optimizers.RMSprop(0.001)
                    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
                    return model
                model = build_model()



                class PrintDot(keras.callbacks.Callback):
                    def on_epoch_end(self, epoch, logs):
                        if epoch % 100 == 0: print('')
                        print('.', end='')

                EPOCHS = j

                history = model.fit(normed_train_data, normed_train_labels,
                                    epochs=EPOCHS, validation_split = 0.2, verbose=0,
                                    callbacks=[PrintDot()])

                test_predictions = model.predict(normed_test_data).flatten()
                unnormed_test_predictions = test_predictions[0]*(max_val_y - min_val_y) + min_val_y

                mape_value = abs(float(test_labels) - unnormed_test_predictions)/float(test_labels)


                result.append({'Neurons':i, 'Epochs':j, 'Function':k, 'MAPE':round(mape_value, 4), 'Prediction':round(unnormed_test_predictions, 4)}, ignore_index=True)
                result_list.append([i, j, k, round(mape_value, 4),round(unnormed_test_predictions, 4)])

                result_df = pd.DataFrame(result_list, columns = ['Neurons', 'Epochs', 'Function', 'MAPE', 'Prediction'] )
                result_df.to_excel(str(l)+'res.xlsx')

